In [2]:
from networks_bf import get_all_devices_combined
from services_bf import get_all_services
import itertools
import numpy as np
import pandas as pd

In [26]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

def preprocess_data(R, S, X, PI, PM, SAL, SIPS, SL):
    # Initialize the scalers
    scaler = MinMaxScaler()
    one_hot_encoder = OneHotEncoder()

    # Normalize the multi-dimensional arrays
    R_norm = scaler.fit_transform(R.flatten().reshape(-1, 1)).reshape(R.shape)
    S_norm = scaler.fit_transform(S.flatten().reshape(-1, 1)).reshape(S.shape)
    X_norm = scaler.fit_transform(X.flatten().reshape(-1, 1)).reshape(X.shape)
    PI_norm = scaler.fit_transform(PI.flatten().reshape(-1, 1)).reshape(PI.shape)
    PM_norm = scaler.fit_transform(PM.flatten().reshape(-1, 1)).reshape(PM.shape)

    # Normalize the one-dimensional arrays
    SAL_norm = scaler.fit_transform(np.array(SAL).reshape(-1, 1)).flatten()
    SIPS_norm = scaler.fit_transform(np.array(SIPS).reshape(-1, 1)).flatten()

    # One-hot encode the categorical array SL
    SL_encoded = one_hot_encoder.fit_transform(np.array(SL).reshape(-1, 1)).toarray()

    # Flatten the arrays and concatenate
    flattened_data = np.concatenate((R_norm.flatten(), S_norm.flatten(), X_norm.flatten(),
                                     PI_norm.flatten(), PM_norm.flatten(), SAL_norm, 
                                     SIPS_norm, SL_encoded.flatten()))

    return flattened_data

In [27]:
inputs = []

# Get the real output values
file_path = 'results_mini_bf.csv'
bf_data = pd.read_csv(file_path)

bf_data['placement_physical'] = bf_data['placement_physical'].apply(lambda x: eval(x))
bf_data['placement_vm'] = bf_data['placement_vm'].apply(lambda x: eval(x))

outputs = bf_data[['placement_physical', 'placement_vm']].values.tolist()

In [28]:
for c in range(1, 101):
    
    print(f"Processing config-{c}....")
    
    physical_machines = get_all_devices_combined(f"config-{c}")
    services = get_all_services(f"config-{c}")
    
    # Define the problem-specific parameters
    n = len(physical_machines)  # Number of physical machines
    s = len(services)  # Number of services
    
    vm_list = []
    
    for p in range(len(physical_machines)):
        for v in range(len(physical_machines[p].guest_machines)):
            vm_list.append([p, v])
    
    # Calculate all permutations of vm_list with s places (with repetition)
    permutations = itertools.product(vm_list, repeat=s)
    
    # Variable number of virtual machines for each physical machine
    max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))
    
    ################ Physical and virtual machine configurations ##################
    
    # Initialization
    R = np.empty(shape=(n, max_v))  # Request network delay of the machine
    R.fill(999999) # Initialize with very large value
    S = np.empty(shape=(n, max_v))  # Response network delay of the machine
    S.fill(999999) # Initialize with very large value
    X = np.empty(shape=(n, max_v))  # Maximum IPS possible
    X.fill(1) # Initialize with 1
    
    PI = np.empty(shape=(n, max_v))  # Power consumption in idle
    PI.fill(999) # Initialize with very large value
    PM = np.empty(shape=(n, max_v))  # Maximum power consumption
    PM.fill(999) # Initialize with very large value
    
    # Fill in with the actual numbers
    for i in range(len(physical_machines)):
        for j in range(len(physical_machines[i].guest_machines)):
            R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
            S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
            X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
            PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
            PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization
    
    ################ Service configurations ###################
    
    # Acceptable latencies for services
    SAL = [service.acceptable_latency for service in services]
    # Initialize IPS values for services (replace with actual data)
    SIPS = [service.average_instructions_per_second for service in services]
    # Layers considered for service
    SL = [service.layer for service in services]
    
    ################# Preprocess the data ####################
    input_data = preprocess_data(R, S, X, PI, PM, SAL, SIPS, SL)
    inputs.append(input_data)

Processing config-1....
Processing config-2....
Processing config-3....
Processing config-4....
Processing config-5....
Processing config-6....
Processing config-7....
Processing config-8....
Processing config-9....
Processing config-10....
Processing config-11....
Processing config-12....
Processing config-13....
Processing config-14....
Processing config-15....
Processing config-16....
Processing config-17....
Processing config-18....
Processing config-19....
Processing config-20....
Processing config-21....
Processing config-22....
Processing config-23....
Processing config-24....
Processing config-25....
Processing config-26....
Processing config-27....
Processing config-28....
Processing config-29....
Processing config-30....
Processing config-31....
Processing config-32....
Processing config-33....
Processing config-34....
Processing config-35....
Processing config-36....
Processing config-37....
Processing config-38....
Processing config-39....
Processing config-40....
Processin

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Convert lists to numpy arrays for TensorFlow compatibility
inputs_array = np.array(inputs)
outputs_array = np.array(outputs)

# Split the data into training and test sets (e.g., 80% training, 20% test)
train_ratio = 0.8
train_size = int(len(inputs_array) * train_ratio)
X_train, X_test = inputs_array[:train_size], inputs_array[train_size:]
y_train, y_test = outputs_array[:train_size], outputs_array[train_size:]

def custom_loss(y_true, y_pred):
    # Define your custom loss computation
    # Example: Mean Squared Error
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    LSTM(16),
    Dense(y_train.shape[1])  # Output layer with a number of neurons equal to the output size
])

model.compile(optimizer='adam', loss=custom_loss)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# After training, use model.predict() on new data
# predictions = model.predict(new_test_data)

# Evaluate the model's performance
model.evaluate(X_test, y_test)

[array([6.64926525e-05, 0.00000000e+00, 4.54245178e-05, 1.00000000e+00,
        1.83867680e-04, 1.00000000e+00, 1.39481631e-03, 1.00000000e+00,
        1.49084447e-04, 2.34485693e-04, 0.00000000e+00, 1.00000000e+00,
        9.09679242e-04, 1.00000000e+00, 3.04320853e-03, 1.00000000e+00,
        3.94419380e-01, 1.00000000e+00, 3.15814403e-01, 0.00000000e+00,
        7.68280752e-01, 0.00000000e+00, 6.52044969e-01, 0.00000000e+00,
        2.99776764e-02, 4.46884028e-04, 1.68560086e-02, 1.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 2.98690874e-02, 1.00000000e+00,
        0.00000000e+00, 2.11952277e-01, 5.17173245e-02, 1.00000000e+00,
        1.95712354e-01, 1.00000000e+00, 1.98417306e-01, 1.00000000e+00,
        0.00000000e+00, 5.30060321e-01, 1.00000000e+00, 9.29504038e-01,
        4.40162370e-01, 9.43867882e-01, 2.18375474e-01, 3.00492352e-02,
        1.00000000e+00, 8.07681829e-01, 2.89587700e-01, 1.42067679e-02,
        8.06658541e-02, 0.00000000e+00, 4.90859940e-01, 4.797225